Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

#open picklefile and save
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
    #print to show that all of 28x28 pixels
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

#reformat data to be 783 0's and 1's where the pixels are
def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
#set train datset/labels to this version, as well as validation dataset and labels as well as test
#***do this because simple logistic regression needs it?
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
#print out to confirm they are all of new shape
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

#initalize tensfor flow graph
graph = tf.Graph()
#make default graph and establish operations
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  # create normal distribution for weights to start since that is efficient
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  # logists are the inputs multiplied by the weights with the biases added
  # loss function is the softmax cross entroy --distnace between scores and 1-hot answers
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  # gradient descent of .5 step? to minimize the loss function
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  # calculate prediictions at each step of the process
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [5]:
#do this for 801 steps
num_steps = 801

#calculate accuracy as # predictions out of # possible
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# uncomment this code to run the logistic regression with gradient descent and no hidden layer
# with tf.Session(graph=graph) as session:
#     tf.global_variables_initializer().run()
#     for step in range(num_steps):
#         _, l, predictions = session.run([optimizer, loss, train_prediction])
#         if (step % 100 == 0):
#             # for every 100th step
#             print('Loss at step %d: %f' %(step, 1))
#             print('Training accuracry: %.lf%%' % accuracy(
#                 predictions, train_labels[:train_subset, :]))
#             print('Validation accuracy: %.lf%%' % accuracy(
#                 valid_prediction.eval(), valid_labels))
#     print('Test accuracy: %.lf%%' % accuracy(test_prediction.eval(), test_labels))
            

#run the tf session
# with tf.Session(graph=graph) as session:
#   # This is a one-time operation which ensures the parameters get initialized as
#   # we described in the graph: random weights for the matrix, zeros for the
#   # biases. 
#   # runs the initalizer we defined above? 
#   tf.global_variables_initializer().run()
#   print('Initialized')
#   for step in range(num_steps):
#     # Run the computations. We tell .run() that we want to run the optimizer,
#     # and get the loss value and the training predictions returned as numpy
#     # arrays.
#     # run session to optimize loss
#     _, l, predictions = session.run([optimizer, loss, train_prediction])
#     if (step % 100 == 0):
#       # print out the loss and training accuracy as we do it
#       print('Loss at step %d: %f' % (step, l))
#       print('Training accuracy: %.1f%%' % accuracy(
#         predictions, train_labels[:train_subset, :]))
#       # Calling .eval() on valid_prediction is basically like calling run(), but
#       # just to get that one numpy array. Note that it recomputes all its graph
#       # dependencies.
#       print('Validation accuracy: %.1f%%' % accuracy(
#         valid_prediction.eval(), valid_labels))
#   print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [11]:
#batch size is # of data points! We're choosing random data which is making
#this more efficient.
batch_size = 128
num_relu = 1024
hidden_2_size = 300

# def weight_variable(shape):
#     initial = tf.truncated_normal(shape, stddev=0.1)
#     return tf.Variable(initial)

# def bias_variable(shape):
#     initial = tf.constant(0.1, shape=shape)
#     return tf.Variable(initial)

# n_hidden_1_features = 256
n_input_features = 784
n_classes = 10
beta = 0.001
beta2 = 0.0001

# initalize graph
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  # create train validation and test data/labels of correct size
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  xavier_initializer = tf.contrib.layers.xavier_initializer()

  # Variables.
  # weights are the truncated normal, biases set to 0
# xavier_initializer([n_input_features, num_relu])
  weights = {
    'h1': tf.Variable(tf.truncated_normal([n_input_features, num_relu])),
    'h2': tf.Variable(tf.truncated_normal([num_relu, hidden_2_size])),
    'out': tf.Variable(tf.truncated_normal([hidden_2_size, n_classes]))
  }

  biases = {
    'h1': tf.Variable(tf.random_normal([num_relu])),
    'h2': tf.Variable(tf.random_normal([hidden_2_size])),
    'out': tf.Variable(tf.random_normal([n_classes]))
  }

#   print('initalizer', xavier_initializer)
#   print('xav weights', xav_weights)
#multilayer perceptron takes in a dataset, weights, and biases
#it runs a normal matmul on it (layer_1)
#then it runs a neural relu layer
#then it runs an out layer to conver to w2
#continuing in stocahstic gradient descent, it uses 128 random vals

  def multilayer_perceptron(x, weights, biases):
        layer_1 = tf.matmul(x, weights['h1']) + biases['h1']
        hidden_layer = tf.nn.tanh(layer_1)
#         hidden_layer = tf.nn.dropout(hidden_layer, .5)
        layer_2 = tf.matmul(hidden_layer, weights['h2']) + biases['h2']
        hidden_2 = tf.nn.tanh(layer_2)
#         hidden_2 = tf.nn.dropout(hidden_2, .5)
#         print('hidden_2', hidden_2)
        out_layer = tf.matmul(hidden_2, weights['out']) + biases['out']
        return out_layer     

#   def multilayer_perceptron_with_dropout(x, weights, biases):
#         layer_1 = tf.matmul(x, weights['h1']) + biases['h1']
#         hidden_layer = tf.nn.relu(layer_1)
#         layer_2 = tf.matmul(hidden_layer, weights['h2']) + biases['h2']
#         hidden_2 = tf.nn.relu(layer_2)
#         out_layer = tf.matmul(hidden_2, weights['out']) + biases['out']
#         return out_layer    
        


#above-- set the training data as placeholder with (batch_size, image_size * image_size) since it is 128 random samples
# of 1-hot encoded info
#set the weights as h1: from input_num (image_size * image_size) to num_relu
#and out: from num_relu to num_classes
        


#   def multilayer_perceptron(x, weights, biases):
#       layer_1 = tf.matmul(x, weights['h1']) + biases['h1']
#       print('layer shape is', layer_1)
#       hidden_layer_1 = tf.nn.relu(layer_1)
#       out_layer = tf.matmul(hidden_layer_1, weights['out']) + biases['out']
#       print('x is', x)
#       print('hidden layer 1 shape', hidden_layer_1)
#       return out_layer


#set logit to multilyaer_perceptron(train, weights, biases)
#set loss to tf.reduce mean of softmax cross entropy of logit and labels 
#initialize optimizer for gradient descent
#run on each datapoint
#what i should do on thursday after getting all this to run is try to rewrite everything 
#without looking back

  # calculate loss
  # Training computation.
  #add in layer_1 instead of train dataset...?
    #old: tf train dataset
  logits = multilayer_perceptron(tf_train_dataset, weights, biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + beta*(tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['h2'])) + beta2*(tf.nn.l2_loss(weights['out']))) 
  
  # Optimizer.
  # set gradient descent-- where is stochastic? 
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(multilayer_perceptron(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(multilayer_perceptron(tf_test_dataset, weights, biases))

Let's run it:

In [12]:
# do it for 3001 operations 
num_steps = 30001

# run tensfor flow session
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #make the batch either the first batch, second, or third
#     if step % 3 == 0: 
#         offset = 0
#     elif step % 3 == 1: 
#         offset = 256
#     else: 
#         offset = 1000
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 454.887787
Minibatch accuracy: 10.9%
Validation accuracy: 11.1%
Minibatch loss at step 500: 264.250977
Minibatch accuracy: 78.9%
Validation accuracy: 72.9%
Minibatch loss at step 1000: 161.692184
Minibatch accuracy: 71.9%
Validation accuracy: 75.1%
Minibatch loss at step 1500: 97.622452
Minibatch accuracy: 85.2%
Validation accuracy: 78.8%
Minibatch loss at step 2000: 59.422127
Minibatch accuracy: 86.7%
Validation accuracy: 80.9%
Minibatch loss at step 2500: 36.401653
Minibatch accuracy: 82.8%
Validation accuracy: 82.6%
Minibatch loss at step 3000: 22.448366
Minibatch accuracy: 81.2%
Validation accuracy: 81.9%
Minibatch loss at step 3500: 13.945829
Minibatch accuracy: 79.7%
Validation accuracy: 83.5%
Minibatch loss at step 4000: 8.493986
Minibatch accuracy: 88.3%
Validation accuracy: 85.2%
Minibatch loss at step 4500: 5.423935
Minibatch accuracy: 86.7%
Validation accuracy: 85.1%
Minibatch loss at step 5000: 3.659275
Minibatch accuracy: 87.5%
Validat

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [ ]:
got to 93.8 by running it 10x as many times! woooooot